# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 7-8. Понятие API. Работа с API ВКонтакте.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**
1. [Введение в API](#par1)
2. [API ВКонтакте: документация и получение доступа](#par2)
3. [Как отправить запрос к API и обработать json](#par3)
4. [Немного о docstrings](#par4)
5. [Собираем данные о наших друзьях](#par5)
6. [Генерируем данные для сетевого анализа](#par6)
7. [Заключение](#parlast)
---

**Основные ссылки**
- [Документация API ВКонтакте](https://dev.vk.com/ru/api/overview);
- [Как получить токен для работы с API ВКонтакте](https://github.com/lika1kapustina/hse_polit_web-scraping/blob/main/api_vk_get_token_and_test.ipynb).

## Введение в API<a name="par1"></a>

В рамках презентации мы с вами подробно обсудили понятие **API**. Для тех, кто пропустил этот материал – **API – Application Programming Interface** – это интерфейс, позволяющий вам взаимодействовать с приложением или веб-сервером. 

<font color='blue'>Например, представим что вы студент, которому нужно забронировать аудиторию для проведения консультации. Делать вы это будете через учебный офис. В этой ситуации:</font>
* **Вы – студент – <font color='green'>пользователь</font>**;
* **<font color='orange'>Сервер</font>, к которому вы бы хотели обратиться – это корпоративные системы университета**;
* **<font color='violet'>API</font>**, которое позволяет вам отправить запрос к серверу и получить данные в удобном формате – **учебный офис**. Вас не беспокоит в каком формате происходит взаимодействие <font color='violet'>API</font> и <font color='orange'>сервера</font> (через какие внутренние сервисы учебный офис будет проверять доступность аудиторий и бронировать для вас её), вам нужно получить только ответ от <font color='violet'>API</font> – то есть, от учебного офиса: письмо о том, что аудитория найдена и забронирована.
* **Взаимодействие с API ограничивается прописанными методами**: вы можете попросить учебный офис прислать вам расписание, принять вашу справку об уважительной причине пропуска занятий, забронировать вам аудиторию для консультации или отписать вас от факультатива. Но у сотрудников учебного офиса есть ряд рабочих обязанностей – это как методы у API, и они не будут делать то что выходит за их пределы. *Например, вы не можете попросить сотрудников учебного офиса отменить вам пересдачу, выставить оценку выше реальной или прислать вам мем*.

**Особенности API, которые мы сейчас с вами увидим:**
1. Запросы к API строго унифицированы;
2. Перед тем как отправить запрос к API, чаще всего нужно получить токен;
3. API отдает вам данные в основном в форматах JSON и XML.

## API ВКонтакте: документация и получение доступа <a name="par1"></a>

**API ВКонтакте: документация**

Социальная сеть ВКонтакте имеет очень много сервисов для разработчиков, в том числе, понятный и хорошо задокументированный API. Документация API ВКонтакте доступна по [ссылке](https://dev.vk.com/ru/api/overview).

**API ВКонтакте имеет множество семейств методов**: `Apps`, `Account`, `Adds`, `Photos` и другие. В чем содержательный смысл? Определенное семейство методов отвечает за взаимодействие с определенными объектами – приложениями, аккаунтом, рекламой, фотографиями и т.д.

**Например**: обращаясь к семейству методов [`Groups`](https://dev.vk.com/ru/method/groups), вы узнаете о методах работы с группами: вы можете получить список сообществ из выбранной категории каталога с помощью метода `groups.getCatalog`, с помощью метода `groups.get` получить список сообществ определенного пользователя, с помощью метода `groups.getById` получить информацию об одном или нескольких сообществах по их id.
<p></p>
<center><b><font size=4>Задача №1</font></b></center>

**Давайте попрактикуемся в работе с API ВКонтакте. Найдите следующие методы и сохраните значения в переменные `answer_1`, `answer_2`, `answer_3`:**
1. Метод, с помощью которого можно **сделать пост** на вашей стене;
2. Метод, с помощью которого можно **получить детальную информацию** об опросе;
3. Метод, с помощью которого можно **получить информацию о подписчиках пользователя**

In [192]:
answer_1 = 'wall.post'
answer_2 = 'polls.getById'
answer_3 = 'users.getFollowers'


**В чем особенности API ВКонтакте?**
1. Это API готово отдать вам огромное количество самых разных данных о разных объектах;
2. Позволяет не просто получать данные из социальной сети, а и управлять группами или своей страницей (пролайкать чужие посты, написать кому-то в сообщения, забанить и т.д.);
3. Имеет хорошую и подробную документацию;
4. Поддержка отвечает в течение суток или двух;

**В чем ограничения API ВКонтакте?**
1. Вы можете получить доступ только к тем данным, к которым имеете доступ в приложении. Не получиться залезть в чужие переписки, получить список друзей пользователя с закрытой страницей и т.д.;
2. Есть [ограничения на число запросов](https://dev.vk.com/ru/api/api-requests#:~:text=К%20методам%20API%20ВКонтакте%20(за,составляет%2020%20запросов%20в%20секунду.): **не больше 3 запросов в секунду**;


**Что нужно, чтобы пользоваться API ВКонтакте?** Нужно получить специальный токен, который позволит ВКонтакте понимать, что запросы отправляете вы и что вы можете получать только определенные данные в ответ (например, вы можете запросить список сообщений, но должны получить только сообщения, отправленные вами, а не сообщения из переписок которые вели другие пользователи). Инструкция по получению токена доступна по [ссылке](https://github.com/lika1kapustina/hse_polit_web-scraping/blob/main/api_vk_get_token_and_test.ipynb).

**Как выглядит токен ВКонтакте?**
По структуре он выглядит так, где:
`vk1.533bacf01e1165b57531ad114461ae8736d6506a3ууу1e2texpires_in=864000id=1111111`
* `vk1.533bacf01e1165b57531ad114461ae8736d6506a3ууу1e2texpires_in=864000id=1111111` – это ваш токен;
* `expires_in=86400` это число секунд, через которое токен потеряет силу (86400 секунд это 24 часа, то есть этот токен действует 24 часа);
* `id=1111111` это ваш id пользователя;

<font color='red'>Важно:</font> если вы сгенерировали токен и собираетесь получать данные из ВКонтакте, не пользуйтесь VPN-сервисами и не меняйте IP-адрес. Если вы включите VPN, то наткнетесь на ошибку `'User authorization failed: access_token was given to another ip address'`. 
* Хорошая новость в том, что если его отключить, то вы сможете продолжать пользоваться вашим токеном и отправлять запросы к ВКонтакте. 
* Если вы поменяли IP-адрес, всегда можно перегенерировать токен, сохранить его в переменную, и продолжить работу.

<font color='red'>Важно: НИКОМУ НЕ ПРИСЫЛАЙТЕ СВОЙ ТОКЕН.</font> ВКонтакте ласково напомнит вам об этом в процессе его получения, но запомните – если вы отдадите кому-то свой токен, то можете потерять доступ к аккаунту.

**Когда вы получили токен, сохраните его в переменную в ноутбуке/файле `.py` из которого будете отправлять запросы к API ВКонтакте**.

## Как отправить запрос к API<a name="par3"></a>

Каждый наш запрос к API ВКонтакте будет выглядеть следующим образом:

0. **Если мы работаем с новым методом ВКонтакте, то первым делом изучаем справку по нему**: какие параметры можно включить в запрос, какой объект этот метод вернет:
    * Например, мы решили собрать посты из сообщества с помощью метода `wall.get`. Документация к нему доступна по [ссылке](https://dev.vk.com/ru/method/wall.get);
    * Там же мы видим раздел **Результат**:
>```После успешного выполнения возвращает объект, содержащий число результатов в поле count и массив объектов записей на стене в поле items. Если был задан параметр extended = 1, возвращает число результатов в поле count, отдельно массив объектов записей на стене в поле items, пользователей в поле profiles и сообществ в поле groups```
    * Помимо этого, на странице с методом мы **в конце увидим, информацию об объектах какого типа он возвращает**. Там же содержится ссылка на описание этого типа объектов. Для `wall.get` это **записи на стене**. Вы можете кликнуть на [ссылку, ведущую на описание этого типа объектов](https://dev.vk.com/ru/reference/objects/post) и увидите там следующее: ```Объект, описывающий запись на стене пользователя или сообщества, содержит следующие поля``` (и перечисленные поля и возможные значения.
    
    

1. **Подбираем параметры**. <font color='blue'>Эти параметры отличаются для разных методов</font>. На странице с методом `wall.get` указаны параметры, которые нам нужно/можно скормить методу. А именно:
    * `owner_id` – Идентификатор пользователя или сообщества, со стены которого необходимо получить записи (по умолчанию — текущий пользователь);
    * `domain` – Короткий адрес пользователя или сообщества;
    * `offset` – Смещение, необходимое для выборки определённого подмножества записей;
    * `count` – Количество записей, которое необходимо получить. Максимальное значение: `100` (то есть, за один запрос мы можем получить только 100 записей);
    * `filter` – Определяет, какие типы записей на стене необходимо получить. Возможные значения: `owner` (записи владельца стены), `others` (записи не от владельца стены) и т.д.;
    * `extended` – Если равен `1` — в ответе будут возвращены дополнительные поля `profiles` и `groups`, содержащие информацию о пользователях и сообществах. По умолчанию: `0`.

2. **Создаем ссылку для запроса:** <font color='green'>Этот шаблон универсален для всех методов</font>. Как она выглядит? В общем виде так:


>```url = 'https://api.vk.com/method/' + method +'?'+ parameters + '&v=' + version + '&access_token=' + token```

**Где:**
* `method` - метод (например, `wall.get`);
* `parameters` – параметры запроса для этого метода (см. пункт №1);
* `version` – версия API ВКонтакте;
* `token` – ваш токен.

**Как уточнить параметры запроса?**

Давайте посмотрим на эту строчку:

>```parameters=f'owner_id=-{group_id}&count=5&filter=owner&extended=1&offset={offset}&fields=name,screen_name,members_count'```

**Где все переменные это параметры из пункта №1:**
* `owner_id` это `Идентификатор пользователя или сообщества, со стены которого необходимо получить записи (по умолчанию — текущий пользователь)`;
* `count=5` означает что мы хотим получить 5 постов;
* `filter=owner` означает что мы хотим получить только посты от имени сообщества;
* `extended=1` означает что мы хотим получить дополнительную информацию по полям `profiles` и `groups`;
* `offset=` означает сдвиг для получения постов;
* `fields=` означает переменные которые мы хотим получить для сообществ.


Здесь мы используем f-строку, не пугайтесь. Напомню синтаксис f-строк в ячейке ниже.

3. **Отправляем запрос с помощью `requests.get()` и обрабатываем информацию!**

In [ ]:
# когда вы используете f-строки, вы можете удобнее создавать и печатать строки. Например:
name = input('Введите ваше имя') # вводим значение

print('Ваше имя это', name) # печать без f-строки
print(f'Выше имя это {name}') # начинаем строку с f, открываем кавычки, открываем фигурные скобки когда хотим 
# напечатать какую-то конкретную переменную

Чтобы начать, давайте импортируем необходимые библиотеки

In [193]:
import requests      # запросы к серверу
import pandas as pd  # работа с таблицами
import numpy as np   # работа с арифметическими операциями
import json          # работа с файлами json

Введем сюда наши данные (хватит только токена)

In [ ]:
token = input('Введите ваш токен тут: ') # вводим токен;
myid = token.split('user_id=')[1]        # получим ваш id пользователя из токена;
version = '5.199'                        # версия api вк, которую вы используете

Сгенерируем строку, чтобы получить посты сообщества [Уполномоченный по правам студентов и аспирантов НИУ ВШЭ](https://vk.com/ombudsman_hse), используя метод `wall.get`:

In [196]:
method = 'wall.get'           # метод
group_id = '203966578'        # id групы
offset = 0                    # сдвиг
q = 5                         # число постов, которые нам вернет этот метод

# Создаем строку с параметрами;
parameters=f'owner_id=-{group_id}&count={q}&filter=owner&extended=1&offset={offset}&fields=name,screen_name,members_count'

# Генерируем ссылку 
url = 'https://api.vk.com/method/' + method +'?'+ parameters + '&v=' + version + '&access_token=' + token

# Отправляем запрос:
response = requests.get(url) # привет, requests
wall_data = response.json() # превращаем ответ в объект json, сохраняем в переменную wall_data

Перед тем, как мы посмотрим на объект `wall_data`, давайте посмотрим на ссылку `url`, которую мы создали. Что же там находится?

In [50]:
print(f'Откройте ссылку: {url}')

Верно, там находится ровно та же самая информация, что и в `wall_data`. **Отличие в том,** что мы обработали эти данные с помощью `requests.get()`, преобразовали в **json**, и сохранили в переменную.

Теперь давайте исследуем объект `wall_data`:

In [197]:
wall_data # посмотрим на объект

{'response': {'count': 92,
  'items': [{'inner_type': 'wall_wallpost',
    'can_delete': 1,
    'can_pin': 1,
    'donut': {'is_donut': False},
    'is_pinned': 1,
    'comments': {'count': 0},
    'marked_as_ads': 0,
    'short_text_rate': 0.8,
    'hash': 'btsjLvwY7QErOnCxhHaG439TyGal',
    'has_translation': False,
    'postponed_id': 468,
    'type': 'post',
    'carousel_offset': 0,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1703315938,
       'id': 457239480,
       'owner_id': -203966578,
       'access_key': '723ffb2b329aab3986',
       'sizes': [{'height': 75,
         'type': 's',
         'width': 75,
         'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=75x75&quality=95&sign=e02501c46310b8866db249785430ff3b&c_uniq_tag=73php1pSyosrCipg3ZrWMqtf3VFcIgnwTEgbm4O4Cqc&type=album'},
        {'height': 130,
         'type': 'm',
         'width': 130,
         'url': 'https://sun9-27.us

In [198]:
wall_data.keys() # ключи, которые есть у объекта - только response

dict_keys(['response'])

In [203]:
wall_data['response'].keys() # что внутри объекта response?

dict_keys(['count', 'items', 'profiles', 'groups', 'reaction_sets'])

Давайте исследуем эти объекты. Как к ним обратиться? Вызвать квадратные скобки и написать их название внутри. 

Как когда вы хотите получить значение из словаря по названию ключа:

In [199]:
# Представим, что мы с вами работаем со словарем, который имеет следующую структуру:
# ключ - это номер группы; значение - это список кураторов группы.
my_dictionary = {'202': ['Жестовская', 'Атабегашвили'],
                 '203': ['Капустина',  'Кувивчак', 'Музыка'],
                '204': ['Крипайтис', 'Белоклокова']}

# Как получить список кураторов группы (значение)? Обратиться к номеру группы (ключу):
my_dictionary['203']

['Капустина', 'Кувивчак', 'Музыка']

То же самое мы делаем когда работаем с json объектами:

In [204]:
wall_data['response']['count'] # 92 - это число постов на стене всего.
# даже если мы запросили всего три последних поста, мы все равно получим этот параметр

92

In [205]:
wall_data['response']['groups'] # информация о группах, которые являются авторами постов которые мы получили

[{'id': 203966578,
  'members_count': 1421,
  'name': 'Уполномоченный по правам студентов НИУ ВШЭ',
  'screen_name': 'ombudsman_hse',
  'is_closed': 0,
  'type': 'page',
  'photo_50': 'https://sun6-20.userapi.com/s/v1/ig2/z62AU5hRrRQI1kzlpqzsAJtYgGhLpAB0itKNjepO2gbSCPVZZbQjFtQeYvBiIxUh0L3kccrmZAuVQdoNxeqVyBhQ.jpg?size=50x50&quality=96&crop=0,227,1819,1819&ava=1',
  'photo_100': 'https://sun6-20.userapi.com/s/v1/ig2/P7494n-87s2HwRHVN4kotnYyHpfB88P3-Q0tjQ2M8Jc1E0NYXOB2es0vxmCm9kYIUIinOsxNB6CdjasilE8aPoxx.jpg?size=100x100&quality=96&crop=0,227,1819,1819&ava=1',
  'photo_200': 'https://sun6-20.userapi.com/s/v1/ig2/SK8b0Ewhf9BJ-2xegajAIfSWScN5w-JHMcjdGGvi7-LEFG8qvuA9VSk1O2mkyhElbtY6kDO8ZoHeOxCEncQuaBui.jpg?size=200x200&quality=96&crop=0,227,1819,1819&ava=1'},
 {'id': 64952366,
  'members_count': 10946,
  'name': 'Студенческий совет НИУ ВШЭ',
  'screen_name': 'hsecouncil',
  'is_closed': 0,
  'type': 'page',
  'photo_50': 'https://sun6-23.userapi.com/s/v1/ig2/LgJ-T2hUc4AipDZkNss-0ckMmgS4OH3V

In [206]:
wall_data['response']['reaction_sets'] # объект с информацией о реакциях;

[{'id': 'reactions',
  'items': [{'id': 0,
    'title': 'Нравится',
    'asset': {'animation_url': 'https://vk.com/reaction/3-reactions-0?c_uniq_tag=83b2081a8e4adfc36ee536f5f1b4ad470174c89678369a4b9dc5547614a3955e',
     'images': [{'url': 'https://vk.com/reaction/1-reactions-0-32?c_uniq_tag=464ba6bdc06e9f204a9b2c865a046355d835f601d8d82be4dc77e43a028741ff',
       'width': 32,
       'height': 32},
      {'url': 'https://vk.com/reaction/1-reactions-0-48?c_uniq_tag=e6bee176471af6e4f7ca0f57ac099847d57b8101bf07944e47b42b097a6d8455',
       'width': 48,
       'height': 48},
      {'url': 'https://vk.com/reaction/1-reactions-0-72?c_uniq_tag=d6f55fe94c0add8f817b447cbd804768752eb996c06ef222b39eb3fd35834780',
       'width': 72,
       'height': 72},
      {'url': 'https://vk.com/reaction/1-reactions-0-96?c_uniq_tag=0a64c3d34d3a1368b05716ff24f94ff51b2257a2287957423ced36a00b020cb6',
       'width': 96,
       'height': 96},
      {'url': 'https://vk.com/reaction/1-reactions-0-144?c_uniq_tag=47

А теперь посмотрим на самый интересный объект - `items`.

Что здесь находится? Здесь находится информация о постах.

In [207]:
wall_data['response']['items'] 

[{'inner_type': 'wall_wallpost',
  'can_delete': 1,
  'can_pin': 1,
  'donut': {'is_donut': False},
  'is_pinned': 1,
  'comments': {'count': 0},
  'marked_as_ads': 0,
  'short_text_rate': 0.8,
  'hash': 'btsjLvwY7QErOnCxhHaG439TyGal',
  'has_translation': False,
  'postponed_id': 468,
  'type': 'post',
  'carousel_offset': 0,
  'attachments': [{'type': 'photo',
    'photo': {'album_id': -7,
     'date': 1703315938,
     'id': 457239480,
     'owner_id': -203966578,
     'access_key': '723ffb2b329aab3986',
     'sizes': [{'height': 75,
       'type': 's',
       'width': 75,
       'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=75x75&quality=95&sign=e02501c46310b8866db249785430ff3b&c_uniq_tag=73php1pSyosrCipg3ZrWMqtf3VFcIgnwTEgbm4O4Cqc&type=album'},
      {'height': 130,
       'type': 'm',
       'width': 130,
       'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=130x130&quality

**Как превратить объект json в табличные данные?**

Для начала отметим, что json файлы поддерживают индексацию. Это значительно упрощает нам жизнь, потому что позволяет пользоваться как циклами, так и генераторами списков:

In [215]:
wall_data['response']['items'][0] # обратимся к индексу.

{'inner_type': 'wall_wallpost',
 'can_delete': 1,
 'can_pin': 1,
 'donut': {'is_donut': False},
 'is_pinned': 1,
 'comments': {'count': 0},
 'marked_as_ads': 0,
 'short_text_rate': 0.8,
 'hash': 'btsjLvwY7QErOnCxhHaG439TyGal',
 'has_translation': False,
 'postponed_id': 468,
 'type': 'post',
 'carousel_offset': 0,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1703315938,
    'id': 457239480,
    'owner_id': -203966578,
    'access_key': '723ffb2b329aab3986',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=75x75&quality=95&sign=e02501c46310b8866db249785430ff3b&c_uniq_tag=73php1pSyosrCipg3ZrWMqtf3VFcIgnwTEgbm4O4Cqc&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=130x130&quality=95&sign=7e5e32786874551cf4

**Как получить конкретные значения для первого поста?**

Чтобы понять какие вообще параметры у нас относятся к конкретному посту, давайте посмотрим на ключи этого json:

In [216]:
wall_data['response']['items'][0]

{'inner_type': 'wall_wallpost',
 'can_delete': 1,
 'can_pin': 1,
 'donut': {'is_donut': False},
 'is_pinned': 1,
 'comments': {'count': 0},
 'marked_as_ads': 0,
 'short_text_rate': 0.8,
 'hash': 'btsjLvwY7QErOnCxhHaG439TyGal',
 'has_translation': False,
 'postponed_id': 468,
 'type': 'post',
 'carousel_offset': 0,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1703315938,
    'id': 457239480,
    'owner_id': -203966578,
    'access_key': '723ffb2b329aab3986',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=75x75&quality=95&sign=e02501c46310b8866db249785430ff3b&c_uniq_tag=73php1pSyosrCipg3ZrWMqtf3VFcIgnwTEgbm4O4Cqc&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun9-27.userapi.com/impg/PEwP6Hikm8w0hpDToWS8mvvBoiQX3cTnpjK3tQ/FfKEepH9gRA.jpg?size=130x130&quality=95&sign=7e5e32786874551cf4

In [217]:
wall_data['response']['items'][0].keys() # все параметры одного поста

dict_keys(['inner_type', 'can_delete', 'can_pin', 'donut', 'is_pinned', 'comments', 'marked_as_ads', 'short_text_rate', 'hash', 'has_translation', 'postponed_id', 'type', 'carousel_offset', 'attachments', 'date', 'edited', 'from_id', 'id', 'is_favorite', 'likes', 'reaction_set_id', 'reactions', 'owner_id', 'post_type', 'reposts', 'text', 'views'])

In [220]:
wall_data['response']['items'][0]['views']['count']

3276

Чтобы получить текст поста, нужно обратиться к первому элементу из `items` в нашем json: ```wall_data['response']['items'][0]```, а далее вызвать нужный атрибут: `['text']` (где `[0]` это индекс первого поста):

In [53]:
wall_data['response']['items'][0]['text'] # получим текст первого поста

'Выборы-выборы-выборы! 🗳 \n \nГолосование заканчивается уже 25 декабря, а вы еще не проголосовали? Самое время это сделать! При этом обратите внимание на тех, кто уже принимал участие в развитии студенческого самоуправления и получил большой опыт в работе локальным омбудсменом или членом Аппарата Уполномоченного🙋 \n \nУзнайте, кто защищает Ваши права и голосуйте на выборах в студенческое самоуправление за них: \n \n🔹 Арсен Алексанян, ФКИ, защищает права студентов ОП «Реклама и связи с общественностью» - https://vk.com/wall-130898791_1181 \n🔸 Терентий Ващекин, ФП, локальный омбудсмен ОП «Право» - https://vk.com/wall-130899003_1382 \n🔹 Аня Старовойтова, ФГН, защищает права студентов ОП «Философия» - https://vk.com/wall-130897171_962 \n🔸 Данила Сергеев, ФГН, защищает права 1-2 курса ОП «Философия» и «Широкого бакалавриата» - https://vk.com/wall-130897171_961 \n🔹 Максим Осипенко, ФГРР, локальный омбудсмен ОП «Городское планирование» - https://vk.com/wall-130930090_306 (а еще ЖК Level Амурс

И так далее:

In [221]:
wall_data['response']['items'][0]['views'] # число просмотров

{'count': 3276}

In [222]:
wall_data['response']['items'][0]['views']['count'] # чтобы получить конкретную цифру, нужно обратиться к параметру count

3276

In [224]:
wall_data['response']['items'][0]['likes']

{'can_like': 0, 'count': 38, 'user_likes': 0}

In [61]:
wall_data['response']['items'][0]['likes'] # число лайков

{'can_like': 0, 'count': 38, 'user_likes': 0}

In [225]:
wall_data['response']['items'][0]['likes']['count'] # число лайков от пользователей

38

А как собрать все эти данные вместе? Хорошо, что у нас с вами есть генераторы списков (если вы забыли как они работают, посмотрите [заполненный ноутбук к семинарам 5-6](https://github.com/lika1kapustina/hse_polit_web-scraping/blob/main/web-scraping_seminar5-6-with_answers.ipynb)). Наши с вами `item` в `items` индексируются, что значит, что мы можем обратиться к конкретному параметру, просто поменяв индекс в квадратных скобках:

In [66]:
# Здесь и далее мы используем вот этот кусочек. Что он значит?
# Это число элементов, которое лежит у нас в items - оно совпадает с запросом, который мы делали (5 постов)
len(wall_data['response']['items'])

5

In [67]:
# использование функции range() создать последовательность от 0 до 4 включительно:
range(len(wall_data['response']['items']))

range(0, 5)

In [68]:
# наглядное подтверждение:
for i in range(len(wall_data['response']['items'])):
    print(i) # эти индексы мы и будем в дальнейшем использовать чтобы разобраться с 

0
1
2
3
4


Итак, соберем в один список просмотры на постах:

In [72]:
# Вариант один - просто с помощью цикла:

for i in range(len(wall_data['response']['items'])): # проходимся по индекс (от 0 до 4 включительно)
    print(wall_data['response']['items'][i]['views']) # печатаем то что содержится под ключом 'views'

{'count': 3273}
{'count': 473}
{'count': 646}
{'count': 985}
{'count': 528}


In [73]:
# чтобы получить точное число просмотров, обращаемся к ключу 'count'
for i in range(len(wall_data['response']['items'])):
    print(wall_data['response']['items'][i]['views']['count'])

3273
473
646
985
528


Другой вариант - используя генераторы списков. 

**Расшифруем запись ниже:**

* Смотрим с конца: ```for i in range(len(wall_data['response']['items']))```:

    * Кусочек ```len(wall_data['response']['items']``` возвращает число объектов `items` – то есть, сколько постов (в данном примере) у нас есть – а это целое число **5**;
    * Кусочек ```range(len(wall_data['response']['items']))``` возвращает `range(0, 5)` - то есть, последовательность от 0 включительно до 5 невключительно - `[0, 1, 2, 3, 4]`;
    * Когда мы используем цикл и проходимся по последовательности `range(0, 5)`, это позволяет нам в дальнейшем вызывать конкретные элементы (смотрим дальше).
    
    
    
* А теперь на кусочек ```wall_data['response']['items'][i]['views']['count']```:
    * Что такое `wall_data`? Это наш объект json, куда мы сохранили наш объект;
    * Что лежит в `wall_data['response']`? Ответ API с разными ключами: если вы запустите строчку кода ```wall_data['response'].keys()```, то увидите, что там есть такие ключи – ```['count', 'items', 'profiles', 'groups', 'reaction_sets']```;
    * Чтобы получить именно `items` - то есть некоторые объекты - мы обращаемся к соответствующему ключу. Получается `wall_data['response']['items']` – и этот код возвращает нам словарь с элементами (то есть, с постами);
    * **Чтобы получить конкретный пост**, мы используем запись ```wall_data['response']['items'][i]['views']``` где `i` - это конкретный индекс элемента в `items`. 
        * Чтобы понять, сколько всего элементов у вас в `items`, воспользуйтесь кодом ```len(wall_data['response']['items']```.
    * **Чтобы получить конкретный атрибут конкретного поста**, используйте запись вида ```[wall_data['response']['items'][i]['views']``` где `['views']` это один конкретный атрибут.
        * Чтобы понять, какие атрибуты вы можете получить из вашего элемента, обратитесь к документации или используйте запись ```wall_data['response']['items'][i].keys()``` где `i` - индекс конкретного элемента;
        * Лучше ориентироваться на документацию и быть готовым к тому, что не во всех элементах будет та информация которую вы хотите получить (например, если вы собираете 1000 последних постов по поисковому запросу в разделе "новости" (метод `newsfeed.search`), вам могут попасться посты сообществ у которых включено отображение просмотров и не включено. Вызов `['views']['count']` может вызвать ошибку. В таких ситуациях надо вспомнить про конструкцию `try-except`.

In [228]:
for i in range(5):
    print(i)

0
1
2
3
4


In [233]:
wall_data['response']['items'][0].keys()

dict_keys(['inner_type', 'can_delete', 'can_pin', 'donut', 'is_pinned', 'comments', 'marked_as_ads', 'short_text_rate', 'hash', 'has_translation', 'postponed_id', 'type', 'carousel_offset', 'attachments', 'date', 'edited', 'from_id', 'id', 'is_favorite', 'likes', 'reaction_set_id', 'reactions', 'owner_id', 'post_type', 'reposts', 'text', 'views'])

In [231]:
[wall_data['response']['items'][i]['views']['count'] for i in range(len(wall_data['response']['items']))] # просмотры

[3276, 479, 651, 988, 529]

<p></p>
<center><b><font size=4>Задача №2</font></b></center>


Давайте соберем `pandas.DataFrame` с информацией о тех пяти постах, которые есть у нас в `wall_data`. Используя генераторы списков или циклы, сделайте следующее:

1. **Создайте следующие списки:**
    * `owner_ids` - список со значениями атрибута `owner_id` (кто запостил запись);
    * `dates` - список с датами публикаций каждого поста (*внутри будут странные целые числа, пока не обращайте внимание*);
    * `texts` - список с текстом каждого поста;
    * `likes` - список с числом лайков на каждом посте (целые числа);
    * `reposts` - список с числом репостов каждого поста.

In [247]:
# YOUR CODE HERE

owner_ids = [wall_data['response']['items'][i]['owner_id'] for i in range(len(wall_data['response']['items']))]
dates = [wall_data['response']['items'][i]['date'] for i in range(len(wall_data['response']['items']))]
texts = [wall_data['response']['items'][i]['text'] for i in range(len(wall_data['response']['items']))]
likes = [wall_data['response']['items'][i]['likes']['count'] for i in range(len(wall_data['response']['items']))]
reposts = [wall_data['response']['items'][i]['reposts']['count'] for i in range(len(wall_data['response']['items']))]

2. **После того как списки созданы, создайте `posts` - `pandas.DataFrame` с информацией об этих постах:**

Как создаются датафреймы в `pandas` можно посмотреть в [официальной документации](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Вы также можете запустить ячейку с кодом ```help(pd.DataFrame)``` или познакомиться с примером ниже:

In [100]:
# синтаксис внутри мягких скобок: {'название колонки': значения, которые там будут}

small_df = pd.DataFrame({'name_of_first_column': [1, 2, 3, 4],
                        'name_of_second_column': ['value', 'value', 'value', 'value']})
small_df # пример

,name_of_first_column,name_of_second_column
0,1,value
1,2,value
2,3,value
3,4,value


 Данные должны выглядеть вот так :) 

In [248]:
posts_new = pd.DataFrame({'owner_id': owner_ids,
                         'date': dates,
                         'text': texts,
                         'likes': likes,
                         'reposts': reposts})
posts_new

,owner_id,date,text,likes,reposts
0,-203966578,1703315788,Выборы-выборы-выборы! 🗳 \n \nГолосование закан...,38,2
1,-203966578,1704051000,Дорогие вышкинцы!\n\nНаша команда хочет поздра...,13,0
2,-203966578,1703669400,Дорогие вышкинцы! \n \nСегодня мы подробно раз...,10,3
3,-203966578,1703496600,"Дорогие вышкинцы!\n \nСегодня мы, совестно с П...",25,7
4,-203966578,1703414444,Уполномоченный по правам студентов и аспиранто...,22,3


In [95]:
# YOUR CODE HERE
posts

,owner_id,date,text,likes,reposts
0,-203966578,1703315788,Выборы-выборы-выборы! 🗳 \n \nГолосование закан...,38,2
1,-203966578,1704051000,Дорогие вышкинцы!\n\nНаша команда хочет поздра...,13,0
2,-203966578,1703669400,Дорогие вышкинцы! \n \nСегодня мы подробно раз...,10,3
3,-203966578,1703496600,"Дорогие вышкинцы!\n \nСегодня мы, совестно с П...",25,7
4,-203966578,1703414444,Уполномоченный по правам студентов и аспиранто...,22,3


3. **Теперь, когда мы написали код, который обрабатывает полученный json в датафрейм с информацией о постах, давайте с вами напишем функцию `get_last_posts()`:**

**Алгоритм решения задачи:**
1. Создайте функцию `get_last_posts()`. Пропишите в скобках, какие параметры она будет принимать:
    * `group_id` - id сообщества, посты которого мы хотим собрать;
    * `q` - целое число, столько последних постов, сколько мы хотели бы получить (до `100`).
    * `offset=0` – параметр, по умолчанию равный `0` (сдвиг);
    * `method='wall.get'` - параметр, по умолчанию равный `'wall.get'` – название метода, который мы используем.
2. Оставьте внутри тела функции три комментария и по пустой строке после них:
    * `# 1. Отправка запроса`;
    * `# 2. Обработка данных`;
    * `# 3. Создание датафрейма`.
3. Перенесите код выше:
    * Сперва перенесите код в раздел `# 1. Отправка запроса` – тот код, который отвечал за генерацию ссылки: начинался с создания строки `parameters` и заканчивался созданием объекта `wall_data`;
        * При создании строки `parameters` замените значение `5` на переменную, которая содержит в себе число запрашиваемых постов (см. пункт 1) (вспомните синтаксис f-строк);
    * Далее перенесите код в раздел `# 2. Обработка данных` – тот код, который помогал вам сгенерировать вам списки `dates`, `likes`, `reposts`, и прочее. 
    * Далее в раздел `# 3. Создание датафрейма` перенесите код, с помощью которого вы генерировали `pandas.DataFrame`;
    * Верните получившийся `pandas.DataFrame` с помощью ключевого слова `return`.

In [250]:
# YOUR CODE HERE
def get_last_posts(group_id, q, offset=0, method='wall.get'):
    # 1. Отправка запроса
    # Создаем строку с параметрами;
    parameters=f'owner_id=-{group_id}&count={q}&filter=owner&extended=1&offset={offset}&fields=name,screen_name,members_count'

    # Генерируем ссылку 
    url = 'https://api.vk.com/method/' + method +'?'+ parameters + '&v=' + version + '&access_token=' + token

    # Отправляем запрос:
    response = requests.get(url) # привет, requests
    wall_data = response.json() # превращаем ответ в объект json, сохраняем в переменную wall_data
    
    # 2. Обрабокта данных
    owner_ids = [wall_data['response']['items'][i]['owner_id'] for i in range(len(wall_data['response']['items']))]
    dates = [wall_data['response']['items'][i]['date'] for i in range(len(wall_data['response']['items']))]
    texts = [wall_data['response']['items'][i]['text'] for i in range(len(wall_data['response']['items']))]
    likes = [wall_data['response']['items'][i]['likes']['count'] for i in range(len(wall_data['response']['items']))]
    reposts = [wall_data['response']['items'][i]['reposts']['count'] for i in range(len(wall_data['response']['items']))]

    # 3. Создание датафрейма
    posts_new = pd.DataFrame({'owner_id': owner_ids,
                         'date': dates,
                         'text': texts,
                         'likes': likes,
                         'reposts': reposts})
    return posts_new

In [252]:
get_last_posts(group_id=203966578, q=100, offset=0, method='wall.get')

,owner_id,date,text,likes,reposts
0,-203966578,1696258802,Дорогие студенты! \n\n21ого сентября Уполномоч...,9,0
1,-203966578,1696255146,Дорогие вышкинцы! \n\nСегодня мы хотим поделит...,23,0
2,-203966578,1696086001,Дорогие студенты и студентки! \n \nМы завершае...,12,8
3,-203966578,1695481201,Дорогие студенты и студентки! \n \nМы продолжа...,11,4
4,-203966578,1694879955,Дорогие студенты и студентки! \n \nМы продолжа...,14,7
...,...,...,...,...,...
72,-203966578,1645376401,"Друзья, всем привет! 💙 #итогинедели\n\nРассказ...",49,6
73,-203966578,1645117201,Привет! Новости из Питера 💙\n\nНа этой неделе ...,42,8
74,-203966578,1644944401,Pov: готовишь студента к Дисциплинарной комисс...,49,5
75,-203966578,1644843602,"Друзья, всем привет! 💓\n\nПоздравляем вас с Дн...",66,15


In [120]:
# Запустите эту ячейку. У вас должна получиться вот такая таблица:
get_last_posts(group_id=203966578, q=100, offset=0, method='wall.get')

,owner_id,date,text,likes,reposts
0,-203966578,1703315788,Выборы-выборы-выборы! 🗳 \n \nГолосование закан...,38,2
1,-203966578,1704051000,Дорогие вышкинцы!\n\nНаша команда хочет поздра...,13,0
2,-203966578,1703669400,Дорогие вышкинцы! \n \nСегодня мы подробно раз...,10,3
3,-203966578,1703496600,"Дорогие вышкинцы!\n \nСегодня мы, совестно с П...",25,7
4,-203966578,1703414444,Уполномоченный по правам студентов и аспиранто...,22,3
...,...,...,...,...,...
87,-203966578,1645376401,"Друзья, всем привет! 💙 #итогинедели\n\nРассказ...",49,6
88,-203966578,1645117201,Привет! Новости из Питера 💙\n\nНа этой неделе ...,42,8
89,-203966578,1644944401,Pov: готовишь студента к Дисциплинарной комисс...,49,5
90,-203966578,1644843602,"Друзья, всем привет! 💓\n\nПоздравляем вас с Дн...",66,15


<b><font color='green'>Поздравляю! У нас с вами получилось написать функцию, которая получает `n` последних постов со стены сообщества.</font></b>. В теории, ничто не мешает вам в дальнейшем усложнить эту функцию, добавить дополнительные переменные, которые вы будете собирать, "углубить" поиск (используя параметр `offset` начинать собирать больше чем 100 последних постов) и так далее. Но этот пример был нужен чтобы показать вам общую линию работы с API ВКонтакте: **как генерируется запрос**, **как понять какие данные вы можете получить**, **как их обработать** и **как собрать нормальную функцию из отдельных кусочков кода**.

4. <font color='blue'>EXTRA – если вы справились раньше остальных</font><b>. Исследуйте документацию API ВК по типу объектов [запись на стене](https://dev.vk.com/ru/reference/objects/post) и выясните, почему в колонке `date` так странно отражается дата публикации поста. </b>

    <b>Осуществите поиск в интернете и выясните, как с помощью модуля <code>datetime</code> можно осуществить преобразование этого целого числа в адекватную дату. Примените этот код к столбцу `date`.</b>

In [ ]:
# YOUR CODE HERE

## Методы `friends` и немного о docstrings<a name="par4"></a>

В этой части мы с вами соберем данные по вашим друзьям и поговорим о такой вещи, как **docstrings**. 

**Что такое docstrings? Это документация к модулю, функции, классу или методу**. Её главное назначение – давать пользователю понять, что же в этом коде происходит, какие у функции есть аргументы, как выглядит пример её использования. 

**Зачем и когда нужно использовать docstring?**
* **Когда вы пишите большие функции, которые возвращают сложные объекты:**
    * Если вы пишите функцию в пять строчек, которая адекватно называется, пользуется базовыми арифметическими функциями, и возвращает целое число – вам может не понадобиться документация;
    * Если вы пишите функцию в 40 строк кода, которая делает запрос к API, обрабатывает данные, возвращает `pandas.DataFrame` с 15 колонками – вам нужна документация к функции (docstring);
* **Когда вы работаете в команде, собираетесь использовать код больше одного раза и показывать код другим людям**. Банально есть риск, что если вы вернетесь к вашим функциям спустя месяц, вы можете не вспомнить, откуда берутся те или иные значения в возвращаемом результате.
* **Когда вам важно поддерживать порядок в вашем коде.**

В разделе [PEP 257](https://peps.python.org/pep-0257/) описаны некоторые конвенции разработчиков относительно docstrings. Перевод на русский доступен на [Хабре по ссылке](https://habr.com/ru/articles/499358/).

Как создать docstring? Проиллюстрирую на примере:

In [253]:
help(round)

Help on built-in function round in module builtins:

round(number, ndigits=None)
    Round a number to a given precision in decimal digits.
    
    The return value is an integer if ndigits is omitted or None.  Otherwise
    the return value has the same type as the number.  ndigits may be negative.



In [254]:
# создаем функцию
def get_some_info(a, b):
    '''Здесь содержится какая-то информация.''' # открываем по три кавычки с каждой стороны, пишем текст

Если теперь вызвать функцию `help()` по этой функции, мы видим эту самую информацию!

In [255]:
help(get_some_info)

Help on function get_some_info in module __main__:

get_some_info(a, b)
    Здесь содержится какая-то информация.



**Если вы пишите функции для сбора данных из API или с веб-страниц, я советую вам делать следующее**:
* **Создавать docstrings**;
* **Указывать, на какие методы API опирается функция**/с каким типом страниц работает;
* **Описывать параметры**, которые есть у функции (те самые `a` и `b`);
* **Описывать объект, который функция возвращает**;
* **Прописывать в конце docstring пример использования функции**.

**Давайте докрутим нашу функцию `get_last_posts()`, и добавим сюда docstring:**

In [125]:
# представим, что мы спустя долгое время откроем наш ноутбук. 
help(get_last_posts) # в справке мы увидим только параметры функции

Help on function get_last_posts in module __main__:

get_last_posts(group_id, q, offset=0, method='wall.get')



Пропишем только описание:

In [256]:
def get_last_posts(group_id, q, offset=0, method='wall.get'):
    '''Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    и возвращает pandas.DataFrame с информацией об этих постах.'''
    pass


In [257]:
help(get_last_posts)

Help on function get_last_posts in module __main__:

get_last_posts(group_id, q, offset=0, method='wall.get')
    Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    и возвращает pandas.DataFrame с информацией об этих постах.



In [ ]:
get_last_posts()

Добавим информацию об аргументах:

In [126]:
posts

,owner_id,date,text,likes,reposts
0,-203966578,1703315788,Выборы-выборы-выборы! 🗳 \n \nГолосование закан...,38,2
1,-203966578,1704051000,Дорогие вышкинцы!\n\nНаша команда хочет поздра...,13,0
2,-203966578,1703669400,Дорогие вышкинцы! \n \nСегодня мы подробно раз...,10,3
3,-203966578,1703496600,"Дорогие вышкинцы!\n \nСегодня мы, совестно с П...",25,7
4,-203966578,1703414444,Уполномоченный по правам студентов и аспиранто...,22,3


In [258]:
def get_last_posts(group_id, q, offset=0, method='wall.get'):
    '''Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.
    '''
    pass

In [259]:
help(get_last_posts)

Help on function get_last_posts in module __main__:

get_last_posts(group_id, q, offset=0, method='wall.get')
    Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.



Добавим информацию об объекте, который функция возвращает:

In [260]:
def get_last_posts(group_id, q, offset=0, method='wall.get'):
    '''Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.
    Returns:
        > pandas.DataFrame - с информацией о постах:
            : owner_id - int - id сообщества, выпустившего пост;
            : date - int - дата публикации поста в формате unixtime;
            : text - str - текст поста;
            : likes - int - число лайков на посте;
            : reposts - int - число репостов на посте.
    
    '''
    pass

In [261]:
help(get_last_posts)

Help on function get_last_posts in module __main__:

get_last_posts(group_id, q, offset=0, method='wall.get')
    Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.
    Returns:
        > pandas.DataFrame - с информацией о постах:
            : owner_id - int - id сообщества, выпустившего пост;
            : date - int - дата публикации поста в формате unixtime;
            : text - str - текст поста;
            : likes - int - число лайков на посте;
            : reposts - int - число репостов на посте.



Добавим пример использования:

In [262]:
def get_last_posts(group_id, q, offset=0, method='wall.get'):
    '''Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.
    Returns:
        > pandas.DataFrame - с информацией о постах:
            : owner_id - int - id сообщества, выпустившего пост;
            : date - int - дата публикации поста в формате unixtime;
            : text - str - текст поста;
            : likes - int - число лайков на посте;
            : reposts - int - число репостов на посте.
    Example:
        >>> example = get_last_posts(group_id=203966578, q=100, offset=0, method='wall.get')
        >>> example
    
    '''
    pass

<font color='green'>Вуаля! Теперь запросим справку по функции:</font>

In [263]:
help(get_last_posts)

Help on function get_last_posts in module __main__:

get_last_posts(group_id, q, offset=0, method='wall.get')
    Функция get_last_posts() принимает на вход id сообщества, число постов (до 100), которые надо вернуть,
    обращается к методу wall.get API ВК и возвращает pandas.DataFrame с информацией об этих постах.
    Arguments:
        > group_id - int - id сообщества (без минуса в начале);
        > q - int - число последних записей, информацию о которых нужно вернуть (до 100);
        > offset=0 - int - смещение постов;
        > method='wall.get' - str - название метода, используемого в функции.
    Returns:
        > pandas.DataFrame - с информацией о постах:
            : owner_id - int - id сообщества, выпустившего пост;
            : date - int - дата публикации поста в формате unixtime;
            : text - str - текст поста;
            : likes - int - число лайков на посте;
            : reposts - int - число репостов на посте.
    Example:
        >>> example = get_last_po

А теперь давайте напишем функции, которые будут собирать даные о друзьях и друзьях наших друзей. 

## Собираем данные о наших друзьях<a name="par5"></a>

Зачем это дело нужно? А мы с вами попытаемся создать сеть своих друзей и понять, из каких сообществ она состоит:

<p></p>
<center><b><font size=4>Задача №3</font></b></center>

Познакомьтесь с документацией метода `friends.get`, доступной по [ссылке](https://dev.vk.com/ru/method/friends.get).

**Сгенерируйте строку `parameters`, запрос (`url`), и сохраните результат обработки запроса в переменную `response`.**

**Учтите следующие параметры:**
* API должно возвращать информацию о максимально возможном числе друзей (найдите на странице: 1) как должен называться соответствующий параметр; и 2) какое максимально возможное число объектов пользователей метод может вернуть);
* Список друзей должен быть отсортирован по алфавитному порядке;

*Подсказка:* если вы указываете несколько параметров в `parameters`, они должны быть разделены знаком `&`. Пример:
```firstparameter=somevalue&secondparameter=somevalue```.

**В результате у вас должно получиться похожий объект, где `676` это число друзей, а значения в `items` - id ваших друзей ВКонтакте**

```
{'response': {'count': 676,
  'items': [463008298,
   511785586,
   53414043,
   501118101,
   338357646,
   343197585,
   693641278,
   203816538,
   689200024,
   64564860]}}```
   

In [264]:
# СТРОКА С ПАРАМЕТРАМИ
# СТРОКА С СОЗДАНИЕМ ССЫЛКИ
# ОТПРАВЛЯЕМ ЗАПРОС
parameters=f'count=500&order=name'
method = 'friends.get'
# Генерируем ссылку 
url = 'https://api.vk.com/method/' + method +'?'+ parameters + '&v=' + version + '&access_token=' + token

# Отправляем запрос:
response = requests.get(url) # привет, requests
friends_data = response.json() # превращаем ответ в объект json, сохраняем в переменную wall_data

In [268]:
friends_data['response']['items']

[463008298,
 511785586,
 53414043,
 501118101,
 338357646,
 343197585,
 693641278,
 203816538,
 689200024,
 64564860,
 434439412,
 436070330,
 225054933,
 385407304,
 306684264,
 559703139,
 382097528,
 96389248,
 617724146,
 300293854,
 382979302,
 252667147,
 237774842,
 469186618,
 551917168,
 235249693,
 170456007,
 229952864,
 374312900,
 425429966,
 562435389,
 357249335,
 188058932,
 260857567,
 245402649,
 359827731,
 350579968,
 819177382,
 357379132,
 211810890,
 800349051,
 671432786,
 556798826,
 270184602,
 2416234,
 253300723,
 365184281,
 229405692,
 74023038,
 150333556,
 257956576,
 176786299,
 159825308,
 288076167,
 10741524,
 397207853,
 52442885,
 47402039,
 157793137,
 821924911,
 254993988,
 232366362,
 382773391,
 33626945,
 111940612,
 160254892,
 254061067,
 310660072,
 190750807,
 200219603,
 105872275,
 116029220,
 131139397,
 203148481,
 345892406,
 430131834,
 232010456,
 560283396,
 264205295,
 336938384,
 200771928,
 118384656,
 181158751,
 172447691,
 4

**Теперь:**
* Сохраните id ваших друзей в переменную `my_friends`. 
* Напечатайте длину этого списка.
* Напечатайте 5 первых idшников ваших друзей.


*У вас должно получиться что-то такое:*
```676
[463008298, 511785586, 53414043, 501118101, 338357646]```

In [270]:
my_friends = friends_data['response']['items']
print(len(my_friends))
print(my_friends[:5])

500
[463008298, 511785586, 53414043, 501118101, 338357646]


**Супер. Теперь расширим нашу функцию.**

**Запросим больше данных о наших друзьях:**

* Добавим в `parameters` дополнительные параметры: уточним, что нам нужна расширенная информация по пользователям `&extended=1` и что нам нужна следующая информация про пользователей: `bdate,sex,domain`

In [271]:
parameters += f'&extended=1&fields=bdate,sex,domain' # уточняем параметры

Ниже добавьте код, который вы использовали в последней задаче (только без `parameters`).

Он должен вернуть информацию о числе ваших друзей `{'count': n}` и ваших друзьях. В моем случае это выглядит так:

```
{'response': {'count': 676,
  'items': [{'id': 463008298,
    'domain': 'al___lex',
    'bdate': '17.8',
    'track_code': 'e34d35e77p4tUxB3UJk-W0pbAyeX3b_xPziMprhtI-kSYRWLpWOD9SJiKHUCnGpZcsOK5Q22zOUmNJvI3B58',
    'sex': 1,
    'first_name': 'Alexandra',
    'last_name': 'Oorzhak',
    'can_access_closed': True,
    'is_closed': False},```

In [272]:
# строку parameters мы уже создали
method = 'friends.get' # наш метод

# Генерируем ссылку 
url = 'https://api.vk.com/method/' + method +'?'+ parameters + '&v=' + version + '&access_token=' + token
response = requests.get(url) # привет, requests
friends_data = response.json() # превращаем ответ в объект json, сохраняем в переменную wall_data

In [273]:
friends_data # дожен получиться такой объект с информацией о наших друзьях

{'response': {'count': 676,
  'items': [{'id': 463008298,
    'domain': 'al___lex',
    'bdate': '17.8',
    'track_code': '22bfb2baxkxyrbwrkRjFaU1GO_zSXFfcB90ReRR42BHui6hl07-rJ33P0ymXSMBgdefQRTE3JMge0QYXcAuH',
    'sex': 1,
    'first_name': 'Alexandra',
    'last_name': 'Oorzhak',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 511785586,
    'domain': 'quindicosafare',
    'bdate': '7.4.2004',
    'track_code': '1a783628cKU3vrxoH5-i5VNykzWO_BKLUPt_5wsjRpA1xhDqj84dzmjbgW9ImqDuZ9R5gm2XYZ9J92iJb1AZ',
    'sex': 1,
    'first_name': 'Alina',
    'last_name': 'Ell',
    'can_access_closed': True,
    'is_closed': True},
   {'id': 53414043,
    'domain': 'id53414043',
    'bdate': '28.9.2000',
    'track_code': '258ae3b1soSpkCp0qJUnlaGYVRhnlFNKt8fMlbkELNtV1vfL3zDf7_CpEnP6liSUkz-0n-_zM0eg3Kn73Xc',
    'sex': 1,
    'first_name': 'Angelika',
    'last_name': 'Zagashvilli',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 501118101,
    'domain': 'anna_oorz

<p></p>
<center><b><font size=4>Задача №4</font></b></center>

**Напишите функцию `get_my_friends()`, которая принимает на вход параметр `method` (используемый метод) и возвращает `pandas.DataFrame` с базовой информацией о ваших друзьях:**

* `id` пользователя;
* `first_name` пользователя;
* `last_name` пользователя;
* `sex` – указанный пол пользователя;
* `is_closed` - профиль пользователя "закрытый" (`True`) или нет (`False`).

    
*Подсказка: выведите на экран переменную `parameters`. Поместите её в тело функции*.

In [274]:
parameters

'count=500&order=name&extended=1&fields=bdate,sex,domain'

In [275]:
def get_my_friends(method='friends.get'):
    # 1. Отправляем запрос
    parameters = 'count=5000&order=name&extended=1&extended=1&fields=bdate,sex,domain'
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token
    response = requests.get(url)
    response = response.json()
    
    # 2. Собираем данные
    ids = [response['response']['items'][i]['id'] for i in range(len(response['response']['items']))]
    first_names = [response['response']['items'][i]['first_name'] for i in range(len(response['response']['items']))]
    last_names = [response['response']['items'][i]['last_name'] for i in range(len(response['response']['items']))]
    sex_of_users = [response['response']['items'][i]['sex'] for i in range(len(response['response']['items']))]
    is_closed_profiles = [response['response']['items'][i]['is_closed'] for i in range(len(response['response']['items']))]
    
    # 3. Соберите pandas.DataFrame
    df = pd.DataFrame({'friend_id': ids,
                      'first_name': first_names,
                      'last_name': last_names,
                      'sex': sex_of_users,
                      'is_closed': is_closed_profiles})
    return df

In [276]:
my_friends = get_my_friends() # функция должна выводить датафрейм с вашими друзьями
my_friends

,friend_id,first_name,last_name,sex,is_closed
0,463008298,Alexandra,Oorzhak,1,False
1,511785586,Alina,Ell,1,True
2,53414043,Angelika,Zagashvilli,1,False
3,501118101,Anna,Oorzhak,1,True
4,338357646,Anopa,Anopa,1,False
...,...,...,...,...,...
671,245402649,Maxim,Bong,2,False
672,225843054,Анастасия,Анисимова,1,True
673,211058196,Виталий,Чаус,2,True
674,322747261,Ира,Голос,1,False


Давайте обработаем наш датафрейм. Во-первых, создадим переменную `full_name`:

In [277]:
 # склеим значения из двух колонок с помощью пробела
my_friends['full_name'] = my_friends['first_name'] + ' ' + my_friends['last_name']
my_friends['full_name'] # получилась новая колонка

0         Alexandra Oorzhak
1                 Alina Ell
2      Angelika Zagashvilli
3              Anna Oorzhak
4               Anopa Anopa
               ...         
671              Maxim Bong
672     Анастасия Анисимова
673            Виталий Чаус
674               Ира Голос
675            Юля Комарова
Name: full_name, Length: 676, dtype: object

Во-вторых, преобразуем в более понятные значения значения из колонки `sex`:

In [278]:
# используем запись в словаре
# если в переменной sex находится 1, вернется значение "ж"
# если в переменной sex находится 2, вернется значение "м"
my_friends['sex'] = my_friends['sex'].apply(lambda x: {1: 'ж', 2: 'м'}[x]) # используем apply() и анонимную функцию
my_friends['sex'] # смотрим на колонку

0      ж
1      ж
2      ж
3      ж
4      ж
      ..
671    м
672    ж
673    м
674    ж
675    ж
Name: sex, Length: 676, dtype: object

<p></p>
<center><b><font size=4>Задача №5</font></b></center>

**А теперь давайте попрактикуемся в написании docstrings: напишите docstring к нашей функции. Сделайте следующее:**
* Укажите её общее описание: что она делает, к чему обращается, что возвращает (в одно предложение);
* На какой метод API опирается;
* Какие аргументы принимает;
* Что возвращает (и какие атрибуты есть у этого объекта);
* Пример использования функции.

In [280]:
def get_my_friends(method='friends.get'):
    '''Функция обращается к методу friends.get API ВК, получает данные по вашим друзьям
    и возвращает pandas.DataFrame с информацией о ваших друзьях.
    
    Args:
    > method='friends.get'
    Returns:
    > pandas.DataFrame с информацией о ваших друзьях:
        > friend_id : int - id вашего друга вконтакте;
        > first_name : str - имя вашего друга вконтакте;
        > last_name : str - фамилия вашего друга вконтакте;
        > sex : int - отображаемый пол (1 - ж, 2 - м);
        > is_closed : bool - закрыт профиль у человека или нет (True - да, False - нет)
    Example:
        >>> get_my_friends() 
    '''
    # Ваша docstring - тут!
    
    # Ваш код функции - тут!
    # 1. Отправляем запрос
    parameters = 'count=5000&order=name&extended=1&extended=1&fields=bdate,sex,domain'
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token
    response = requests.get(url)
    response = response.json()
    
    # 2. Собираем данные
    ids = [response['response']['items'][i]['id'] for i in range(len(response['response']['items']))]
    first_names = [response['response']['items'][i]['first_name'] for i in range(len(response['response']['items']))]
    last_names = [response['response']['items'][i]['last_name'] for i in range(len(response['response']['items']))]
    sex_of_users = [response['response']['items'][i]['sex'] for i in range(len(response['response']['items']))]
    is_closed_profiles = [response['response']['items'][i]['is_closed'] for i in range(len(response['response']['items']))]
    
    # 3. Соберите pandas.DataFrame
    df = pd.DataFrame({'friend_id': ids,
                      'first_name': first_names,
                      'last_name': last_names,
                      'sex': sex_of_users,
                      'is_closed': is_closed_profiles})
    return df

In [281]:
help(get_my_friends) # вот такую справку можно написать по вашей функции:

Help on function get_my_friends in module __main__:

get_my_friends(method='friends.get')
    Функция обращается к методу friends.get API ВК, получает данные по вашим друзьям
    и возвращает pandas.DataFrame с информацией о ваших друзьях.
    
    Args:
    > method='friends.get'
    Returns:
    > pandas.DataFrame с информацией о ваших друзьях:
        > friend_id : int - id вашего друга вконтакте;
        > first_name : str - имя вашего друга вконтакте;
        > last_name : str - фамилия вашего друга вконтакте;
        > sex : int - отображаемый пол (1 - ж, 2 - м);
        > is_closed : bool - закрыт профиль у человека или нет (True - да, False - нет)
    Example:
        >>> get_my_friends()



## Генерируем данные для сетевого анализа<a name="par6"></a>

Ниже я предлагаю вам воспользоваться уже написанными мной функциями для сбора данных по друзьям и кратко поясню, что происходит и как вы можете сгенерировать сетевые данные с помощью API ВКонтакте:

In [169]:
from time import sleep # чтобы делать паузы
import tqdm # прогресс-бар

In [170]:
def get_id_of_user_from_link(user_link, method='util.resolveScreeenName'):
    '''
    Функция get_id_of_user_from_link(user_link) принимает на вход ссылку на страницу пользователя ВКонтакте
    и возвращает его id.
    
    Args:
        :user_link: string - ссылка на пользователя;
    Returns:
        :user_id: string - id пользователя в виде строки
    Example:
        >>>> one_user_id = get_id_of_user_from_link('https://vk.com/lika.kapustina')
        >>>> one_user_id # 441721976
    '''

    screen_name = user_link.replace('https://vk.com/', '')

    # может быть такое, что ссылка на пользователя содержит id (vk.com/id777)
    if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
        user_id = screen_name.replace('id', '')

    # если нет, нам нужно сделать запрос к ВК и получить id пользователя
    else:
        method_for_get_id = 'utils.resolveScreenName'
        parameters='screen_name=' + screen_name
        url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
        response = requests.get(url)
        response = response.json()
        user_id = response['response']['object_id']
        
    # возвращаю id
    return user_id


def get_id_of_all_users_links(users_links, method='util.resolveScreeenName'):
    '''
    Функция get_ids_of_all_users_from_links(user_links) принимает на вход СПИСОК со ссылками на аккаунты пользователей,
    обращается внутри себя к методу `get_id_of_user_from_link(user_link)`,
    и возвращает СТРОКУ с id пользователей, записанных через строку.
    
    Args:
        :users_links: list - список с ссылками на пользователей;
    Returns:
        :users_ids_string: string - id пользователей, записанные через запятую в виде строки.
    Example:
        >>>> list_with_ids = get_id_of_all_users_links(['https://vk.com/lika.kapustina','https://vk.com/sdilov','https://vk.com/sofiagreseva'])
        >>>> list_with_ids
        
    '''
    users_ids_list = [] # создаем пустой список
    
    # идем по всему списку из ссылок
    for user_link in tqdm.tqdm(users_links):
        user_id = get_id_of_user_from_link(f'{user_link}') # получаем id пользователя
        users_ids_list.append(user_id)
        sleep(0.4) # небольшая пауза    
    
    users_ids_string = ','.join([str(i) for i in users_ids_list])
    
    return users_ids_string # возвращаем строку с id пользователей


def get_friends_of_user_only_ids(user_id, method='friends.get'):
    '''Функция get_friends_of_user_only_ids принимает на вход id пользователя ВКонтакте и возвращает строку, где через
    запятую перечислены id его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте
    Returns:
        :friends_string: string с id друзей пользователя
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    # осуществляем запрос
    parameters = 'count=5000' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + user_id
    response = requests.get(url)
    one_data = response.text
    
    # обрабатываем данные
    first_index = one_data.index('[')
    second_index = one_data.index(']')
    friends_string = one_data[first_index+1:second_index]
    return friends_string # возвращаем строку с id друзей пользователя


def get_friends_of_user_full_info_in_df(user_id, type_of = 'id', method='friends.get'):
    '''Функция get_friends_of_user_info_df принимает на вход ИЛИ id пользователя ИЛИ ССЫЛКУ ВКонтакте 
    и возвращает датафрейм, где перечислены данные его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте;
        :type_of: тип введенных данных (id пользователя ('id')/ссылка на него('link')).
        
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователя.
        
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    
    # если type_of = 'link' (то есть, не id), нужно получить id пользователя
    if type_of != 'id':
        link = user_id #значит, нам попалась ссылка и из нее нужно вынуть screen_name пользователя
        screen_name = link.replace('https://vk.com/', '')
        
        # может быть такое, что ссылка на полььзователя содержит id (vk.com/id777)
        if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
            user_id = screen_name.replace('id', '')
        
        # если нет, нам нужно сделать запрос к ВК и получить id пользователя
        else:
            method_for_get_id = 'utils.resolveScreenName'
            parameters='screen_name=' + screen_name
            url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
            response = requests.get(url)
            response = response.json()
            user_id = response['response']['object_id']

    
    # осуществляем запрос
    parameters = 'count=5000' + '&extended=1' + '&fields=bdate,city,country,sex,domain' + '&name_case=nom' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + str(user_id)
    response = requests.get(url)
    response = response.json()
    
    # собираем данные для датафрейма
    number_of_friends = response['response']['count'] # число друзей у пользователя
    
    # список: чьи друзья в этом датафрейме?
    owner_of_friends = [user_id] * number_of_friends
    
    
    # общая информация, получается без необходимости использовать try-except.
    friends_ids = [response['response']['items'][i]['id'] for i in range(number_of_friends)]
    friends_first_names = [response['response']['items'][i]['first_name'] for i in range(number_of_friends)]
    friends_second_names = [response['response']['items'][i]['last_name'] for i in range(number_of_friends)]
    friends_sexs = [response['response']['items'][i]['sex'] for i in range(number_of_friends)]
    friends_is_closed = [response['response']['items'][i]['is_closed'] for i in range(number_of_friends)]


    
    # информация, которую нужно получать через try-except
    friends_domains = []
    for i in range(number_of_friends):
        try:
            one_domain = response['response']['items'][i]['domain']
        except:
            one_domain = str(response['response']['items'][i]['id'])
        friends_domains.append(one_domain)
        
        
    ## ссылки на пользвателей; если есть domain - значит, с domain; если нет - значит, с id
    friends_links = list(map(lambda x: f'https://vk.com/{x}' if x.isdigit() == False else \
                       f'https://vk.com/id{x}', friends_domains))
        
    # собираем датафрейм
    friends_df = pd.DataFrame({'friend_owner': owner_of_friends,
                               'user_id': friends_ids,
                              'user_name': friends_first_names,
                              'user_surname': friends_second_names,
                              'user_sex': friends_sexs,
                              'user_domain': friends_domains,
                              'user_is_closed': friends_is_closed,
                              'user_link': friends_links})
    return friends_df


def get_friends_of_a_few_users_full_info_in_df(users_ids, type_of='id', method='friends.get'):
    '''Функция get_friends_of_a_few_users_info_df принимает на вход строку с id пользователей ВКонтакте,
    записанными через запятую, обращается к функции get_friends_of_user_full_info_in_df(user_id),
    и возвращает pandas.DataFrame с информацией о всех друзьях всех указанных пользователей
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :users_ids: string: ИЛИ id пользователей ВКонтакте ИЛИ ссылки на страницы пользователей ВКонтакте, записанных через запятую;
        :type_of: string: тип данных на входе (по умолчанию 'id' - id пользователей), либо 'link' - ссылки на пользователей.
    
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователей, чьи id переданы функции.
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> some_friends = get_friends_of_a_few_users_full_info_in_df('48905537,12900237')
        >>>> some_friends
        
    Exmple:
        >>>> my_friends_list = ['https://vk.com/zzimablue',
                                   'https://vk.com/ksperov',
                                   'https://vk.com/sofiagreseva',
                                   'https://vk.com/sdilov',
                                   'https://vk.com/narepapoyan',
                                   'https://vk.com/kirillmuzyka']
        >>>> my_friends_string = ','.join(my_friends_list)
        >>>> friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')
        >>>> friends_of_my_friends
    '''
    
    users_ids = users_ids.split(',')
    
    full_users_friends_data = pd.DataFrame()
    
    for user_id in tqdm.tqdm(users_ids):
        try:
            one_user_friends_data = get_friends_of_user_full_info_in_df(user_id,
                                                                       type_of = type_of)
            full_users_friends_data = pd.concat([full_users_friends_data, one_user_friends_data])
            sleep(0.4)
        except Exception as e:
            print(e)
    
    return full_users_friends_data

Теперь запустим код ниже:

**Сперва введите сюда ссылку на ваш профиль:**

In [ ]:
# вводим ссылку на ваш профиль:
vk_profile_link = input('Введите ссылку на ваш профиль ВКонтакте: ') # моя, например, 'https://vk.com/lika.kapustina'

In [171]:
friends_data = get_friends_of_user_full_info_in_df(vk_profile_link, type_of = 'link') # получаем список друзей

my_friends_list = friends_data['user_link'].tolist() # получаем список друзей наших друзей
my_friends_string = ','.join(my_friends_list)        # склеиваем idшники друзей в строку, разделяемым запятыми
 
# а теперь пользуемся функцией get_friends_of_a_few_users_full_info_in_df - 
# и получаем датафрейм с моими друзьями 
# и друзьями всех друзей моих друзей (людей, которых я знаю лично и через 1 рукопожатие)
friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')

  9%|███▋                                      | 59/676 [01:36<12:30,  1.22s/it]

'response'


 41%|████████████████▌                        | 274/676 [06:37<07:55,  1.18s/it]

'response'


 50%|████████████████████▎                    | 335/676 [08:02<06:23,  1.13s/it]

...

 90%|████████████████████████████████████▉    | 608/676 [13:56<00:51,  1.31it/s]

'response'


 99%|████████████████████████████████████████▍| 667/676 [15:12<00:08,  1.01it/s]

'response'


100%|█████████████████████████████████████████| 676/676 [15:22<00:00,  1.36s/it]


In [175]:
friends_of_my_friends # датафрейм выглядит.. прилично!

,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_is_closed,user_link
0,130896,1450,Елена,Карпова,1,karmannoe_zlo,False,https://vk.com/karmannoe_zlo
1,130896,6487,Михаил,Шлемин,2,id6487,False,https://vk.com/id6487
2,130896,9337,Даниил,Олегович,2,hashmaker,False,https://vk.com/hashmaker
3,130896,11446,Константин,Печенежский,2,konstantin_p,False,https://vk.com/konstantin_p
4,130896,12662,Григорий,Григорьев,2,evil_gr,False,https://vk.com/evil_gr
...,...,...,...,...,...,...,...,...
41,821924911,601140432,Михаил,Иванов,2,mibog108,False,https://vk.com/mibog108
42,821924911,612597498,Дмитрий,Котов,2,id612597498,False,https://vk.com/id612597498
43,821924911,720390827,Юрий,Слепичев,2,yurimj,False,https://vk.com/yurimj
44,821924911,736387180,Michel,Angelo,2,imshvets,False,https://vk.com/imshvets


Немного обработаем датафрейм, **чтобы там остались только мои друзья, но появились данные о дружеских связях между ними**

In [176]:
my_friends_ids = friends_data['user_id'].tolist() # сохраняем список только моих друзей в список
my_friends_ids.append(friends_data['friend_owner'].unique()[0]) # добавляем в список себя

# создаем small_df, сбрасываем дупликаты
small_df = friends_of_my_friends[['user_id', 'user_name', 'user_surname']].drop_duplicates()
friends_of_my_friends = pd.merge(friends_of_my_friends, small_df, left_on='friend_owner', right_on='user_id')
friends_of_my_friends = friends_of_my_friends[['user_id_x', 'user_name_x', 'user_surname_x', 'user_id_y', 'user_name_y', 'user_surname_y']]

# Создаем колонки с полными именами пользователей
friends_of_my_friends['user_x'] = friends_of_my_friends['user_name_x'] + ' ' + friends_of_my_friends['user_surname_x']
friends_of_my_friends['user_y'] = friends_of_my_friends['user_name_y'] + ' ' + friends_of_my_friends['user_surname_y']

# Оставляем в friends_of_my_friends только тех пользователей, которые есть у вас в друзьях:
friends_of_my_friends = friends_of_my_friends[(friends_of_my_friends['user_id_x'].isin(my_friends_ids)) &\
                     friends_of_my_friends['user_id_y'].isin(my_friends_ids)]
friends_of_my_friends # финальный датафрейм!

,user_id_x,user_name_x,user_surname_x,user_id_y,user_name_y,user_surname_y,user_x,user_y
784,12325849,Андрей,Гречко,130896,Егор,Юрескул,Андрей Гречко,Егор Юрескул
963,38031682,Юлиан,Баландин,130896,Егор,Юрескул,Юлиан Баландин,Егор Юрескул
968,38584695,Федор,Духновский,130896,Егор,Юрескул,Федор Духновский,Егор Юрескул
987,42187308,Иван,Александров,130896,Егор,Юрескул,Иван Александров,Егор Юрескул
994,43907854,Мария,Учаева,130896,Егор,Юрескул,Мария Учаева,Егор Юрескул
...,...,...,...,...,...,...,...,...
296989,202296311,Данила,Морозов,821924911,Александр,Матвиенко,Данила Морозов,Александр Матвиенко
296997,241798539,Николай,Лобов,821924911,Александр,Матвиенко,Николай Лобов,Александр Матвиенко
297002,274066679,Дарья,Рахмалёва,821924911,Александр,Матвиенко,Дарья Рахмалёва,Александр Матвиенко
297013,441721976,Lika,Kapustina,821924911,Александр,Матвиенко,Lika Kapustina,Александр Матвиенко


Давайте сохраним наши данные.

In [180]:
friends_of_my_friends.to_excel('my_friends_network_31_01_2024.xlsx')

А теперь прочитаем их заново и создадим объект `GEXF` для дальнейшей работы в GEFI. Для этого импортируем пакет `networkx`, позволяющий строить сети и работать с сетевыми объектами в Python.

<font color='orange'>Если на моменте импорта `networkx` вы столкнулись с ошибкой</font> ```ImportError: cannot import name 'gcd' from 'fractions' (/opt/anaconda3/lib/python3.9/fractions.py)```, откройте этот ноутбук в Google Colab и продолжите работу там.

In [184]:
import networkx as nx # импортируем пакет networkx как nx для того чтобы работать с сетями

In [ ]:
df = pd.read_excel('my_friends_network_31_01_2024.xlsx') # сохраняем в df наш датафрейм

Тут мы используем функцию `.from_pandas_edgelist()`, которая позволяет создать сеть на основе датафрейма `pandas`, где `user_x` - это пользователь, от которого исходит ребро, а `user_y` - пользователь, к которому идет ребро. Запустим код ниже и сохраним нашу сеть в переменную `G`.

In [ ]:
G = nx.from_pandas_edgelist(df, 'user_x', 'user_y')

In [ ]:
G # сетевой объект

А теперь сохраним эту самую сеть в формате `.gexf`, чтобы дальше мы могли открыть этот файл в Gephi.

In [ ]:
nx.write_gexf(G, "my_friends_network_31_01_2024.gexf") # сохраняем

А теперь давайте откроем **Gephi Lite** – недавно вышедшую онлайн-версию программы Gephi, которую вы можете открыть онлайн.

* Откройте ссылку: https://gephi.org/gephi-lite/ ;
* Выберите `open local graph`, и откройте документ `my_friends_network_31_01_2024.gexf`;
* Откройте `Statistics` и посчитайте `degree` и `Louvian community detection` метрики;
* Откройте `Appearence (node)` и установите цвет в зависимости от модулярности;
* Откройте `Layout` и поэкспериментируйте с позицией сети.

**Обсуждали ли вы модулярность и метод выявления сообществ Лувена?**

## Заключение<a name="parlast"></a>

Сегодня мы с вами обсудили API ВКонтакте. Надеюсь, это было полезно для вас. То, что я советую вам помнить:
* **Если можно использовать API - используйте API**;
* **Запомните способ, с помощью которого генерируются запросы к API ВКонтакте**;
* **Работаешь с API и не понимаешь что делать? Обращайся к документации!**